In [5]:
import nltk
import numpy as np
import pandas as pd
import string
!pip3 install google-cloud-vision
from google.cloud import vision
import io

  Using cached google_cloud_vision-3.4.1-py2.py3-none-any.whl (444 kB)
  Using cached proto_plus-1.22.2-py3-none-any.whl (47 kB)
  Using cached google_api_core-2.11.0-py3-none-any.whl (120 kB)
  Using cached grpcio_status-1.54.0-py3-none-any.whl (5.1 kB)


In [7]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    # print('Texts:')
 
    #area-----------
    max_area = 0
    max_area_name = ""
    text_list = []

    for text in texts:
        text_list.append('\n"{}"'.format(text.description))
        

        vertices = [(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices]

        # vertices = (['({},{})'.format(vertex.x, vertex.y)
        #             for vertex in text.bounding_poly.vertices])
        
        #Case 1: Area using the vertices (Highest area might be the case) - WORKING
        area = 0
        for i in range(len(vertices)):
            x1, y1 = vertices[i]
            x2, y2 = vertices[(i + 1) % len(vertices)]
            area += x1 * y2 - x2 * y1
        area /= 2
        if area > max_area:
            max_area = area
            # max_area_name = text.description

        #Case 2: 

    # text_list.append(max_area_name)
    # print("Name with highest area:", max_area_name)


    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    return text_list

In [21]:
#sqlite database
import sqlite3
conn = sqlite3.connect("mydatabase.db")
c = conn.cursor()
c.execute("SELECT name FROM mytable")
db_names = [row[0].strip() for row in c.fetchall() if row[0].strip()]
db_vecs = np.load("db_vecs.npy")



In [22]:
df = pd.DataFrame(columns=['text'])
df['text'] = db_names

In [23]:
df.head()

,text
0,Augmentin 625 Duo Tablet
1,Azithral 500 Tablet
2,Allegra 120mg Tablet
3,Avil 25 Tablet
4,Allegra-M Tablet


In [24]:
from tqdm import tqdm
def lower_case(df):
    df['text'] = df['text'].apply(str.lower)
    return df

def remove_punctuations(df):
    cleaned_text = []
    for index in tqdm(range(df.shape[0])):
        text = df['text'].iloc[index]

        word_tokens = text.split()
        
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in word_tokens]

        filtered_sentence = " ".join(stripped).strip()
        cleaned_text.append(filtered_sentence)
    df['text'] = np.array(cleaned_text)
    return df

def remove_null(df):
    if df['text'].isnull().sum() > 0:
        df.dropna(inplace = True)
    return df

In [25]:
df = lower_case(df)
df = remove_punctuations(df)
df = remove_null(df)

100%|██████████| 173429/173429 [00:02<00:00, 80378.56it/s]


In [26]:
df.head()

,text
0,augmentin 625 duo tablet
1,azithral 500 tablet
2,allegra 120mg tablet
3,avil 25 tablet
4,allegram tablet
